In [1]:
from torch.utils.data import DataLoader

from src.data_utils import DataUtils
from src.next_token_dataset import NextTokenDataset, tokenizer

# data_utils = DataUtils()
# data_utils.load_and_clean('data/tweets.txt')
# # print(len(data))
# # print(data[:3])
# # # txt.texts_stats()
# data_utils.save_cleaned_data(data_utils.data, 'data/cleaned_tweets.txt')
# # print(data[-1])
# train, val, test = data_utils.split_data(data_utils.data)
# data_utils.save_cleaned_data(train, 'data/train.txt')
# data_utils.save_cleaned_data(val, 'data/val.txt')
# data_utils.save_cleaned_data(test, 'data/test.txt')
# all_texts = NextTokenDataset(data_utils.data, tokenizer, seq_len=3)
# # tw.samples_processing(data, tokenizer)


# train_tokens, val_tokens, test_tokens = data_utils.split_data(all_texts.tokens)

# data_utils.save_tokenized_data_csv(train_tokens, 'data/train_tokens.csv')
# data_utils.save_tokenized_data_csv(val_tokens, 'data/val_tokens.csv')
# data_utils.save_tokenized_data_csv(test_tokens, 'data/test_tokens.csv')
# # tw.samples_processing_from_csv('data/tokens.csv')
# print(all_texts[-1])



In [2]:
# # тренировочный, валидационный и тестовые датасеты из очищенных данных
# train_dataset = NextTokenDataset(train, tokenizer, seq_len=3)
# val_dataset = NextTokenDataset(val, tokenizer, seq_len=3)
# test_dataset = NextTokenDataset(test, tokenizer, seq_len=3)
# print("Datasets complet!")
# # даталоадеры
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64)
# test_loader = DataLoader(test_dataset, batch_size=64)

# print(f"Train dataset size: {len(train_dataset)}, Val dataset size: {len(val_dataset)}, Test dataset size: {len(test_dataset)}")
# print(f"Train loader size: {len(train_loader)}, Val loader size: {len(val_loader)}, Test loader size: {len(test_loader)}")

In [3]:
train = val = test = []
# тренировочный, валидационный и тестовые датасеты из подготовленных токенов
train_dataset = NextTokenDataset(train, tokenizer, seq_len=3, tokens_csv='data/train_tokens.csv')
val_dataset = NextTokenDataset(val, tokenizer, seq_len=3, tokens_csv='data/val_tokens.csv')
test_dataset = NextTokenDataset(test, tokenizer, seq_len=3, tokens_csv='data/test_tokens.csv')
print("Datasets complet!")
# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)
test_loader = DataLoader(test_dataset, batch_size=256)

print(f"Train dataset size: {len(train_dataset)}, Val dataset size: {len(val_dataset)}, Test dataset size: {len(test_dataset)}")
print(f"Train loader size: {len(train_loader)}, Val loader size: {len(val_loader)}, Test loader size: {len(test_loader)}")

Start samples processing from csv -> data/train_tokens.csv
Start samples processing from csv -> data/val_tokens.csv
Start samples processing from csv -> data/test_tokens.csv
Datasets complet!
Train dataset size: 17636044, Val dataset size: 2204484, Test dataset size: 2208055
Train loader size: 68891, Val loader size: 8612, Test loader size: 8626


In [4]:
from src.lstm_model import model, device, count_parameters


param_count = count_parameters(model)
print(f"{param_count:>10,}")

Using device: cuda
20,880,442


In [5]:
from src.eval_lstm import EvalLSTM

evaluator = EvalLSTM(model, device, compute_rouge=True)




In [6]:
from src.lstm_train import LSTMTrain

lstm = LSTMTrain(model, device, evaluator)
lstm.model_train(train_loader, val_loader, tokenizer)

Start model train


Epoch 1: 100%|██████████| 68891/68891 [28:57<00:00, 39.65it/s]


Start evaluate
Epoch 1 | Train Loss: 5.477 | Val Loss: 5.385 | Val Accuracy: 17.02%
| ROUGE-1: 0.0200 | ROUGE-2: 0.0000 | ROUGE-L: 0.0200 | ROUGE-Lsum: 0.0200


Epoch 2: 100%|██████████| 68891/68891 [29:10<00:00, 39.36it/s]  


Start evaluate
Epoch 2 | Train Loss: 5.405 | Val Loss: 5.375 | Val Accuracy: 17.14%
| ROUGE-1: 0.0200 | ROUGE-2: 0.0000 | ROUGE-L: 0.0200 | ROUGE-Lsum: 0.0200


...

Start evaluate
Epoch 3 | Train Loss: 5.398 | Val Loss: 5.369 | Val Accuracy: 17.13%
| ROUGE-1: 0.0200 | ROUGE-2: 0.0000 | ROUGE-L: 0.0200 | ROUGE-Lsum: 0.0200


Epoch 4: 100%|██████████| 68891/68891 [29:34<00:00, 38.83it/s]


Start evaluate
Epoch 4 | Train Loss: 5.395 | Val Loss: 5.367 | Val Accuracy: 17.15%
| ROUGE-1: 0.0000 | ROUGE-2: 0.0000 | ROUGE-L: 0.0000 | ROUGE-Lsum: 0.0000


Epoch 5: 100%|██████████| 68891/68891 [29:31<00:00, 38.89it/s]


Start evaluate
Epoch 5 | Train Loss: 5.394 | Val Loss: 5.364 | Val Accuracy: 17.22%
| ROUGE-1: 0.0200 | ROUGE-2: 0.0000 | ROUGE-L: 0.0200 | ROUGE-Lsum: 0.0200


In [28]:
# токенизируем промпт im just such a big fan of your photography and clothes and im always into modeling so i kinda freaked out a little bit
prompt = "fan of your photography"
input_ids = tokenizer.encode(prompt)

text = evaluator.generate(input_ids, n_tokens=5, tokenizer=tokenizer)
print(text)

and i have to go


In [24]:
# токенизируем промпт 
# prompt = "im just such"
prompt = "fan of your photography"
input_ids = tokenizer.encode(prompt)

text = evaluator.generate_with_sampling(input_ids, n_tokens=10, tokenizer=tokenizer, temperature=0.8, top_k=50)
print(text)

and some of course you should do well its gonna


In [11]:
import torch
torch.save(model, 'models/lstm_512_1layer.pth')

In [30]:
torch.save(model.state_dict(), 'models/lstm_512_1layer_state_dict.pth')
